In [ ]:
#from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
DTYPE = torch.float16 if DEVICE in ("cuda", "mps") else torch.float32
print(f"Using {DEVICE}")

def get_generator(seed):
    if DEVICE in ("cuda", "mps"):
        return torch.Generator(DEVICE).manual_seed(seed)
    generator = torch.Generator()
    generator.manual_seed(seed)
    return generator

#model_name = "Qwen/Qwen2.5-7B-Instruct"
model_name = "Qwen/Qwen2.5-3B-Instruct"
pipe = transformers.AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=DTYPE)
    #torch_dtype="auto",
    #device_map="auto"
if DEVICE in ("cuda", "mps"):
    pipe = pipe.to(DEVICE)

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

def chat(messages):
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    generated_ids = pipe.generate(**model_inputs,max_new_tokens=512)
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

In [ ]:
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
resp = chat(messages)
print(resp)